# Model: 02
> Model Used for Rag             : <strong> command-a-03-2025 </strong><br>

> Embedding Model Used for Rag   : <strong> embed-english-v3.0 </strong><br>

> Model Used for Rag Evaluation      : <strong> llama-3.1-8b-instant </strong>

> Embedding Used for Rag Evaluation      : <strong> embed-english-v3.0 </strong>

In [3]:
!pip install langchain langchain-openai langchain-cohere langchain-mistralai ragas rank-bm25 faiss-cpu langchain-community -qU

In [18]:
import os
os.environ['COHERE_API_KEY'] = 'vcoGZMR2WYJhYKqwE9g18BdLjRgq26CTFCCrXld6'
os.environ['MISTRAL_API_KEY'] = 'FIxsBfuRcIia2LU3hJXdfryqokskW7eO'

In [5]:
from typing import List
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_mistralai import MistralAIEmbeddings
from langchain_cohere import CohereEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain_core.documents import Document

class OptimizedRAGPipeline:
    def __init__(self,
                 docs_path: str = 'doc/',
                 chunk_size: int = 2000,
                 chunk_overlap: int = 200,
                 embedding_model: str = 'embed-english-v3.0',
                 use_reranker: bool = True,
                 use_compression: bool = True):
        """
        Initialize the optimized RAG pipeline

        Args:
            docs_path (str): Path to the documents directory
            chunk_size (int): Size of document chunks
            chunk_overlap (int): Overlap between chunks
            embedding_model (str): Cohere embedding model name (note: uses 'model' parameter in newer versions)
            use_reranker (bool): Whether to use reranking
            use_compression (bool): Whether to use contextual compression
        """
        self.docs_path = docs_path
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.embedding_model = embedding_model
        self.use_reranker = use_reranker
        self.use_compression = use_compression

        # Initialize components
        self.documents = self._load_documents()
        self.embeddings = self._initialize_embeddings()
        self.vector_store = self._create_vector_store()
        self.retriever = self._build_retriever()

    def _load_documents(self) -> List[Document]:
        """Load and split documents from the specified path"""

        try:
            # Load documents from directory
            loader = DirectoryLoader(
                self.docs_path,
                glob='*.txt',
                loader_cls=TextLoader,
                loader_kwargs={"encoding": "utf-8"}
            )

            # Split documents with optimized chunking strategy
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=self.chunk_size,
                chunk_overlap=self.chunk_overlap,
                separators=["\n\n", "\n", ". ", " ", ""],
                length_function=len
            )

            documents = loader.load_and_split(text_splitter)

            return documents

        except Exception as e:

            return []

    def _initialize_embeddings(self):
        """Initialize the embedding model"""

        try:
            return CohereEmbeddings(model=self.embedding_model)
        except Exception as e:
            try:
                return CohereEmbeddings(model=self.embedding_model)
            except Exception as fallback_error:
                return CohereEmbeddings(model=self.embedding_model)

    def _create_vector_store(self):
        """Create a vector store from documents and embeddings"""

        try:
            return FAISS.from_documents(self.documents, self.embeddings)
        except Exception as e:

            return None

    def _build_retriever(self, k: int = 3):
        """Build an optimized retriever with multiple strategies"""

        # Create base vector retriever
        vector_retriever = self.vector_store.as_retriever(search_kwargs={"k": k})

        # Create BM25 retriever for keyword-based retrieval
        bm25_retriever = BM25Retriever.from_documents(self.documents)
        bm25_retriever.k = k

        # Combine retrievers with weights
        ensemble_retriever = EnsembleRetriever(
            retrievers=[vector_retriever, bm25_retriever],
            weights=[0.7, 0.3]
        )

        # Add contextual compression if enabled
        if self.use_compression:
            try:
                try:
                    from langchain_cohere import ChatCohere
                    llm = ChatCohere(
                        cohere_api_key=os.environ.get('COHERE_API_KEY'),
                        temperature=0
                    )
                except Exception:
                    print("Error in contextual compression")

                compressor = LLMChainExtractor.from_llm(llm)

                return ContextualCompressionRetriever(
                    base_compressor=compressor,
                    base_retriever=ensemble_retriever
                )
            except Exception as e:
                return ensemble_retriever
        else:
            return ensemble_retriever

    def retrieve(self, query: str, k: int = 3) -> List[Document]:
        """
        Retrieve relevant documents for a query

        Args:
            query (str): The search query
            k (int): Number of documents to retrieve

        Returns:
            List[Document]: List of retrieved documents
        """

        try:
            # Use the new invoke method instead of deprecated get_relevant_documents
            return self.retriever.invoke(query)
        except Exception as e:

            # Fallback to simple vector retrieval if ensemble fails
            try:
                return self.vector_store.similarity_search(query, k=k)
            except Exception as inner_e:
                return []

    def get_retrieval_content(self, query: str, k: int = 3) -> List[str]:
        """
        Get the content of retrieved documents for a query

        Args:
            query (str): The search query
            k (int): Number of documents to retrieve

        Returns:
            List[str]: List of document contents
        """
        docs = self.retrieve(query, k)
        return [doc.page_content if doc.page_content != 'NO_OUTPUT.' else '' for doc in docs]


# Initialize the RAG pipeline
rag_pipeline = OptimizedRAGPipeline()

In [6]:
retrieve = rag_pipeline.retrieve('What are the services provided by Gokap Inno Tech')
retrieve

[Document(id='45c18669-c6ee-4b83-9db7-fdd762774075', metadata={'source': 'doc\\main_corpus.txt'}, page_content='1. Company Overview\nGOKAP Innotech is a Bhutan-based startup founded to bridge the gap between freelancers and clients through an innovative platform that facilitates seamless collaboration. The company’s mission is "Tech Excellence, Customer Happiness," focusing on transforming lives through technology innovation 4. With over 100+ happy customers and 100+ completed projects, GOKAP has established itself as a trusted name in the tech industry.\n\nFull Name: GOKAP InnoTech (commonly referred to as GIT)\nFounded: 2020\nIndustry: Technology, Freelance Marketplace\nHeadquarters: Thimphu, Bhutan\nContact Information:\nPhone: +97517570958\nEmail: GIT@gokapinnotech.com\nWebsite: www.gokapinnotech.com\n\n2. Platform Description\nGOKAP InnoTech operates a digital marketplace connecting freelancers with clients. The platform facilitates project-based work where clients can post projec

In [8]:
retrieval_content = rag_pipeline.get_retrieval_content('What are the services provided by Gokap Inno Tech')
retrieval_content

['1. Company Overview\nGOKAP Innotech is a Bhutan-based startup founded to bridge the gap between freelancers and clients through an innovative platform that facilitates seamless collaboration. The company’s mission is "Tech Excellence, Customer Happiness," focusing on transforming lives through technology innovation 4. With over 100+ happy customers and 100+ completed projects, GOKAP has established itself as a trusted name in the tech industry.\n\nFull Name: GOKAP InnoTech (commonly referred to as GIT)\nFounded: 2020\nIndustry: Technology, Freelance Marketplace\nHeadquarters: Thimphu, Bhutan\nContact Information:\nPhone: +97517570958\nEmail: GIT@gokapinnotech.com\nWebsite: www.gokapinnotech.com\n\n2. Platform Description\nGOKAP InnoTech operates a digital marketplace connecting freelancers with clients. The platform facilitates project-based work where clients can post projects and freelancers can apply to complete these projects for compensation.\nUser Types\n\nClients:\n\nBusinesse

In [19]:
from langchain_cohere import ChatCohere
llm = ChatCohere()

In [20]:
print(llm.model)

command-a-03-2025


In [21]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
  "You are a helpful customer support assistant for gokap innotech company. "
  "Your goal is to provide accurate, helpful, and concise responses based on the company's knowledge base. "
  Asked Question = {question}
  Retrived Context = {context}
""")

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = rag_pipeline.get_retrieval_content

chain = (
    {"context" : retriever, "question" : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [23]:
answer = chain.invoke("What are the services provided by Gokap Inno Tech")
answer

'Gokap Inno Tech provides a range of services centered around its digital marketplace platform, which connects freelancers with clients. Here are the key services offered:\n\n1. **Freelancer-Client Connection**:  \n   - Facilitates seamless collaboration between freelancers and clients for project-based work.  \n   - Clients can post projects, set budgets, and hire freelancers.  \n   - Freelancers can create profiles, showcase skills, and apply for projects.  \n\n2. **Project Management Tools**:  \n   - Project posting and discovery interfaces.  \n   - Milestone tracking and deadline management features.  \n   - File sharing capabilities for efficient collaboration.  \n\n3. **Secure Payment System**:  \n   - Escrow-based payment processing for secure transactions.  \n   - Payment release upon satisfactory project completion.  \n\n4. **Account Creation & Management**:  \n   - Simple registration with email verification.  \n   - Profile customization and document verification for identit

# Ragas

In [14]:
!pip install ragas -qU

In [15]:
!pip install langchain-groq

In [24]:
from ragas import evaluate

In [25]:
from ragas import SingleTurnSample, EvaluationDataset

In [26]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity

In [27]:
groq_api= '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'
os.environ['GROQ_API_KEY'] = groq_api

In [28]:
cohere_embeddings = CohereEmbeddings(
    cohere_api_key="WbDwiNgZambTJnI3Uwq8nr5LZ5P5AJn8lPWjWnrI",
    model="embed-english-v3.0",
)

In [29]:
from langchain_groq import ChatGroq

groq_llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=100000
)

In [30]:
# Sample 1
user_input = "How do I reset my password on GOKAP InnoTech?"
sample1 = SingleTurnSample(
    user_input=user_input,
    retrieved_contexts=rag_pipeline.get_retrieval_content(user_input),
    response=chain.invoke(user_input),
    reference="To reset your password on GOKAP InnoTech, click on the 'Login' button at the top right of the homepage, select 'Forgot Password', and enter the email address associated with your account. You’ll receive a password reset link via email—be sure to check your spam/junk folder. Click the link and follow the instructions to create a new password. If you don’t receive the email within 10 minutes, contact support@gokapinnotech.com."
)

In [31]:
sample1

SingleTurnSample(user_input='How do I reset my password on GOKAP InnoTech?', retrieved_contexts=['# GOKAP InnoTech - Comprehensive FAQ Answers\n\n## Account Management\n\n### 1. "How do I reset my password?"\nTo reset your password on GOKAP InnoTech:\n1. Click on the "Login" button at the top right of the homepage\n2. Select "Forgot Password" below the login form\n3. Enter the email address associated with your account\n4. Check your email inbox for a password reset link (check spam/junk folders if not visible)\n5. Click the link and follow instructions to create a new password\n6. Use your new password to log in\n\nIf you don\'t receive the reset email within 10 minutes, you can contact our support team at support@gokapinnotech.com for assistance.\n\n### 2. "Can I have both a freelancer and client account?"\nYes, GOKAP InnoTech allows you to maintain both freelancer and client profiles under a single account. To set this up:\n\n1. Log in to your existing account\n2. Go to "Account Set

In [32]:
dataset1 = EvaluationDataset(samples=[sample1])

In [33]:
dataset1

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=1)

In [34]:
result1 = evaluate(
    dataset=dataset1, 
    llm=groq_llm,
    embeddings=cohere_embeddings
)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [35]:
result1

{'answer_relevancy': 0.8078, 'context_precision': 1.0000, 'faithfulness': 1.0000, 'context_recall': 1.0000}

In [36]:
# Sample 2
user_input = "Can I use GOKAP InnoTech as both a freelancer and a client?"
sample2 = SingleTurnSample(
    user_input=user_input,
    retrieved_contexts=rag_pipeline.get_retrieval_content(user_input),
    response=chain.invoke(user_input),
    reference="Yes, GOKAP InnoTech allows users to maintain both freelancer and client profiles under a single account. After logging in, go to 'Account Settings' > 'Profile Management', and click 'Add New Profile Type'. Choose either 'Freelancer' or 'Client' and fill in the required information. Once set up, you can toggle between profiles easily via the top navigation bar."
)

In [37]:
sample2

SingleTurnSample(user_input='Can I use GOKAP InnoTech as both a freelancer and a client?', retrieved_contexts=['1. Company Overview\nGOKAP Innotech is a Bhutan-based startup founded to bridge the gap between freelancers and clients through an innovative platform that facilitates seamless collaboration. The company’s mission is "Tech Excellence, Customer Happiness," focusing on transforming lives through technology innovation 4. With over 100+ happy customers and 100+ completed projects, GOKAP has established itself as a trusted name in the tech industry.\n\nFull Name: GOKAP InnoTech (commonly referred to as GIT)\nFounded: 2020\nIndustry: Technology, Freelance Marketplace\nHeadquarters: Thimphu, Bhutan\nContact Information:\nPhone: +97517570958\nEmail: GIT@gokapinnotech.com\nWebsite: www.gokapinnotech.com\n\n2. Platform Description\nGOKAP InnoTech operates a digital marketplace connecting freelancers with clients. The platform facilitates project-based work where clients can post projec

In [38]:
dataset2 = EvaluationDataset(samples=[sample2])

In [39]:
dataset2

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=1)

In [40]:
result2 = evaluate(
    dataset=dataset2, 
    llm=groq_llm,
    embeddings=cohere_embeddings
)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [43]:
result2

{'answer_relevancy': 0.7745, 'context_precision': 1.0000, 'faithfulness': 0.7500, 'context_recall': 0.2500}

In [44]:
# Sample 3
user_input = "What documents are needed for identity verification?"
sample3 = SingleTurnSample(
    user_input=user_input,
    retrieved_contexts=rag_pipeline.get_retrieval_content(user_input),
    response=chain.invoke(user_input),
    reference="For identity verification, GOKAP InnoTech requires a valid government-issued photo ID (passport, national ID, or driver’s license) and a selfie holding that ID. For address verification, accepted documents include a utility bill or bank statement from the past 3 months, or any official government document showing your address. Business accounts must also submit a business registration certificate, tax ID, and company letterhead or official document. All files must be in JPG, PNG, or PDF format (max 5MB), clearly visible, and in color."
)

In [45]:
sample3

SingleTurnSample(user_input='What documents are needed for identity verification?', retrieved_contexts=['# GOKAP InnoTech - Comprehensive FAQ Answers\n\n## Account Management\n\n### 1. "How do I reset my password?"\nTo reset your password on GOKAP InnoTech:\n1. Click on the "Login" button at the top right of the homepage\n2. Select "Forgot Password" below the login form\n3. Enter the email address associated with your account\n4. Check your email inbox for a password reset link (check spam/junk folders if not visible)\n5. Click the link and follow instructions to create a new password\n6. Use your new password to log in\n\nIf you don\'t receive the reset email within 10 minutes, you can contact our support team at support@gokapinnotech.com for assistance.\n\n### 2. "Can I have both a freelancer and client account?"\nYes, GOKAP InnoTech allows you to maintain both freelancer and client profiles under a single account. To set this up:\n\n1. Log in to your existing account\n2. Go to "Acco

In [48]:
dataset3 = EvaluationDataset(samples=[sample3])

In [49]:
dataset3

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=1)

In [50]:
result3 = evaluate(
    dataset=dataset3, 
    llm=groq_llm,
    embeddings=cohere_embeddings
)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [51]:
result3

{'answer_relevancy': 0.6403, 'context_precision': 1.0000, 'faithfulness': 0.9167, 'context_recall': 0.7143}

In [52]:
# Sample 4
user_input = "How do I temporarily deactivate my GOKAP InnoTech account?"
sample4 = SingleTurnSample(
    user_input=user_input,
    retrieved_contexts=rag_pipeline.get_retrieval_content(user_input),
    response=chain.invoke(user_input),
    reference="To temporarily deactivate your account, log in and go to 'Account Settings', scroll to 'Account Status', and select 'Temporarily Deactivate Account'. Choose a reason from the dropdown menu and specify a deactivation duration (from 1 week to 3 months). During deactivation, your profile won’t appear in searches, you’ll still receive essential emails, and active projects won’t be interrupted. You can reactivate your account anytime by logging in again."
)

In [53]:
sample4

SingleTurnSample(user_input='How do I temporarily deactivate my GOKAP InnoTech account?', retrieved_contexts=['Our team typically reviews verification requests within 1-2 business days. You\'ll receive an email notification once your verification is complete. Verified accounts receive a special badge on their profile, which increases trust with potential clients or freelancers.\n\n### 4. "How can I deactivate my account temporarily?"\nTo temporarily deactivate your GOKAP InnoTech account:\n\n1. Log in to your account\n2. Go to "Account Settings"\n3. Scroll down to "Account Status"\n4. Select "Temporarily Deactivate Account"\n5. Choose a reason for deactivation from the dropdown menu\n6. Specify the duration (1 week to 3 months)\n7. Confirm your decision\n\nDuring deactivation:\n- Your profile won\'t appear in search results\n- Ongoing projects will not be affected\n- You\'ll continue to receive essential emails\n- You can reactivate anytime by logging in again\n\nNote that if you have 

In [54]:
dataset4 = EvaluationDataset(samples=[sample4])

In [55]:
dataset4

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=1)

In [56]:
result4 = evaluate(
    dataset=dataset4, 
    llm=groq_llm,
    embeddings=cohere_embeddings
)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [57]:
result4

{'answer_relevancy': 0.7915, 'context_precision': 1.0000, 'faithfulness': 1.0000, 'context_recall': 1.0000}

In [58]:
# Sample 5
user_input = "How long does identity verification take on GOKAP InnoTech?"
sample5 = SingleTurnSample(
    user_input=user_input,
    retrieved_contexts=rag_pipeline.get_retrieval_content(user_input),
    response=chain.invoke(user_input),
    reference="Identity verification on GOKAP InnoTech typically takes 1–2 business days. Once your documents are reviewed and approved, you will receive an email notification. Verified users get a badge on their profile, which helps build trust with potential clients or freelancers."
)

In [59]:
sample5

SingleTurnSample(user_input='How long does identity verification take on GOKAP InnoTech?', retrieved_contexts=['# GOKAP InnoTech - Comprehensive FAQ Answers\n\n## Account Management\n\n### 1. "How do I reset my password?"\nTo reset your password on GOKAP InnoTech:\n1. Click on the "Login" button at the top right of the homepage\n2. Select "Forgot Password" below the login form\n3. Enter the email address associated with your account\n4. Check your email inbox for a password reset link (check spam/junk folders if not visible)\n5. Click the link and follow instructions to create a new password\n6. Use your new password to log in\n\nIf you don\'t receive the reset email within 10 minutes, you can contact our support team at support@gokapinnotech.com for assistance.\n\n### 2. "Can I have both a freelancer and client account?"\nYes, GOKAP InnoTech allows you to maintain both freelancer and client profiles under a single account. To set this up:\n\n1. Log in to your existing account\n2. Go t

In [60]:
dataset5 = EvaluationDataset(samples=[sample5])

In [61]:
dataset5

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=1)

In [62]:
result5 = evaluate(
    dataset=dataset5, 
    llm=groq_llm,
    embeddings=cohere_embeddings
)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [63]:
result5

{'answer_relevancy': 0.7551, 'context_precision': 0.5000, 'faithfulness': 1.0000, 'context_recall': 1.0000}

In [64]:
# Sample 6
user_input = "What is GOKAP InnoTech and what does the company do?"
sample6 = SingleTurnSample(
    user_input=user_input,
    retrieved_contexts=rag_pipeline.get_retrieval_content(user_input),
    response=chain.invoke(user_input),
    reference="GOKAP InnoTech (GIT) is a Bhutan-based technology startup founded in 2020. It operates a freelance marketplace platform that connects clients and freelancers for project-based work. With a mission of 'Tech Excellence, Customer Happiness,' GOKAP has completed over 100 projects and served more than 100 happy customers. The platform is headquartered in Thimphu, Bhutan, and is known for promoting seamless collaboration through technology."
)

In [65]:
sample6

SingleTurnSample(user_input='What is GOKAP InnoTech and what does the company do?', retrieved_contexts=['1. Company Overview\nGOKAP Innotech is a Bhutan-based startup founded to bridge the gap between freelancers and clients through an innovative platform that facilitates seamless collaboration. The company’s mission is "Tech Excellence, Customer Happiness," focusing on transforming lives through technology innovation 4. With over 100+ happy customers and 100+ completed projects, GOKAP has established itself as a trusted name in the tech industry.\n\nFull Name: GOKAP InnoTech (commonly referred to as GIT)\nFounded: 2020\nIndustry: Technology, Freelance Marketplace\nHeadquarters: Thimphu, Bhutan\nContact Information:\nPhone: +97517570958\nEmail: GIT@gokapinnotech.com\nWebsite: www.gokapinnotech.com\n\n2. Platform Description\nGOKAP InnoTech operates a digital marketplace connecting freelancers with clients. The platform facilitates project-based work where clients can post projects and 

In [66]:
dataset6 = EvaluationDataset(samples=[sample6])

In [67]:
dataset6

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=1)

In [68]:
result6 = evaluate(
    dataset=dataset6, 
    llm=groq_llm,
    embeddings=cohere_embeddings
)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [69]:
result6

{'answer_relevancy': 0.1716, 'context_precision': 1.0000, 'faithfulness': 1.0000, 'context_recall': 1.0000}

In [70]:
# Sample 7
user_input = "What types of services does GOKAP InnoTech offer?"
sample7 = SingleTurnSample(
    user_input=user_input,
    retrieved_contexts=rag_pipeline.get_retrieval_content(user_input),
    response=chain.invoke(user_input),
    reference="GOKAP InnoTech offers a wide range of freelance services across various categories including Web Development (Frontend, Backend, Full-Stack, E-commerce, CMS), Design (UI/UX, Graphic, Logo, Branding, Illustration), Content Creation (Copywriting, Editing, Translation, Proofreading), Digital Marketing (SEO, Social Media, Email Marketing, PPC), and Business Support (Virtual Assistance, Customer Support, Data Entry, Research)."
)

In [71]:
sample7

SingleTurnSample(user_input='What types of services does GOKAP InnoTech offer?', retrieved_contexts=['1. Company Overview\nGOKAP Innotech is a Bhutan-based startup founded to bridge the gap between freelancers and clients through an innovative platform that facilitates seamless collaboration. The company’s mission is "Tech Excellence, Customer Happiness," focusing on transforming lives through technology innovation 4. With over 100+ happy customers and 100+ completed projects, GOKAP has established itself as a trusted name in the tech industry.\n\nFull Name: GOKAP InnoTech (commonly referred to as GIT)\nFounded: 2020\nIndustry: Technology, Freelance Marketplace\nHeadquarters: Thimphu, Bhutan\nContact Information:\nPhone: +97517570958\nEmail: GIT@gokapinnotech.com\nWebsite: www.gokapinnotech.com\n\n2. Platform Description\nGOKAP InnoTech operates a digital marketplace connecting freelancers with clients. The platform facilitates project-based work where clients can post projects and fre

In [72]:
dataset7 = EvaluationDataset(samples=[sample7])

In [73]:
dataset7

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=1)

In [74]:
result7 = evaluate(
    dataset=dataset7, 
    llm=groq_llm,
    embeddings=cohere_embeddings
)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [75]:
result7

{'answer_relevancy': 0.7905, 'context_precision': 0.0000, 'faithfulness': 1.0000, 'context_recall': 1.0000}

In [76]:
# Sample 8
user_input = "What is the mission and vision of GOKAP InnoTech?"
sample8 = SingleTurnSample(
    user_input=user_input,
    retrieved_contexts=rag_pipeline.get_retrieval_content(user_input),
    response=chain.invoke(user_input),
    reference="GOKAP InnoTech’s mission is to provide top-notch services that foster learning opportunities for individuals and businesses while driving technological innovation. The company’s vision is to create a lasting impact on the global technology sector by offering innovative solutions, facilitating education, and expanding its influence through larger institutions."
)

In [77]:
sample8

SingleTurnSample(user_input='What is the mission and vision of GOKAP InnoTech?', retrieved_contexts=['1. Company Overview\nGOKAP Innotech is a Bhutan-based startup founded to bridge the gap between freelancers and clients through an innovative platform that facilitates seamless collaboration. The company’s mission is "Tech Excellence, Customer Happiness," focusing on transforming lives through technology innovation 4. With over 100+ happy customers and 100+ completed projects, GOKAP has established itself as a trusted name in the tech industry.\n\nFull Name: GOKAP InnoTech (commonly referred to as GIT)\nFounded: 2020\nIndustry: Technology, Freelance Marketplace\nHeadquarters: Thimphu, Bhutan\nContact Information:\nPhone: +97517570958\nEmail: GIT@gokapinnotech.com\nWebsite: www.gokapinnotech.com\n\n2. Platform Description\nGOKAP InnoTech operates a digital marketplace connecting freelancers with clients. The platform facilitates project-based work where clients can post projects and fre

In [78]:
dataset8 = EvaluationDataset(samples=[sample8])

In [79]:
dataset8

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=1)

In [80]:
result8 = evaluate(
    dataset=dataset8, 
    llm=groq_llm,
    embeddings=cohere_embeddings
)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [100]:
result8

{'answer_relevancy': 0.7522, 'context_precision': 1.0000, 'faithfulness': 0.8000, 'context_recall': 1.0000}

In [81]:
# Sample 9
user_input = "Are there any success stories or testimonials from GOKAP users?"
sample9 = SingleTurnSample(
    user_input=user_input,
    retrieved_contexts=rag_pipeline.get_retrieval_content(user_input),
    response=chain.invoke(user_input),
    reference="Yes, GOKAP InnoTech has received several positive testimonials from users. Vince Alce, a freelancer, praised the platform for the opportunities it provided. Elisha Eboh, CEO of MOI, highlighted the platform’s efficiency in connecting clients with the right talent. Jay Son Lozada, a software developer, commended GOKAP’s UI design expertise in transforming ideas into user-friendly interfaces."
)

In [82]:
sample9

SingleTurnSample(user_input='Are there any success stories or testimonials from GOKAP users?', retrieved_contexts=['1. Company Overview\nGOKAP Innotech is a Bhutan-based startup founded to bridge the gap between freelancers and clients through an innovative platform that facilitates seamless collaboration. The company’s mission is "Tech Excellence, Customer Happiness," focusing on transforming lives through technology innovation 4. With over 100+ happy customers and 100+ completed projects, GOKAP has established itself as a trusted name in the tech industry.\n\nFull Name: GOKAP InnoTech (commonly referred to as GIT)\nFounded: 2020\nIndustry: Technology, Freelance Marketplace\nHeadquarters: Thimphu, Bhutan\nContact Information:\nPhone: +97517570958\nEmail: GIT@gokapinnotech.com\nWebsite: www.gokapinnotech.com\n\n2. Platform Description\nGOKAP InnoTech operates a digital marketplace connecting freelancers with clients. The platform facilitates project-based work where clients can post pr

In [83]:
dataset9 = EvaluationDataset(samples=[sample9])

In [84]:
result9 = evaluate(
    dataset=dataset9, 
    llm=groq_llm,
    embeddings=cohere_embeddings
)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [107]:
result9

{'answer_relevancy': 0.6321, 'context_precision': 1.0000, 'faithfulness': 0.8571, 'context_recall': 0.2500}

In [85]:
# Sample 10
user_input = "What should I do if I'm not satisfied with the freelancer's work on GOKAP InnoTech?"
sample10 = SingleTurnSample(
    user_input=user_input,
    retrieved_contexts=rag_pipeline.get_retrieval_content(user_input),
    response=chain.invoke(user_input),
    reference="If you're not satisfied with the freelancer's work on GOKAP InnoTech, the platform recommends starting with direct communication through the project messaging system. If the issue persists, you can request revisions by selecting 'Request Revisions' under the project details. For unresolved disputes, you can open a formal dispute via 'Project Actions' and submit supporting evidence. GOKAP’s mediation team will review the case within 48 hours. The platform also offers options for partial payments or full refunds, depending on the situation and project type."
)

In [86]:
sample10

SingleTurnSample(user_input="What should I do if I'm not satisfied with the freelancer's work on GOKAP InnoTech?", retrieved_contexts=['1. Company Overview\nGOKAP Innotech is a Bhutan-based startup founded to bridge the gap between freelancers and clients through an innovative platform that facilitates seamless collaboration. The company’s mission is "Tech Excellence, Customer Happiness," focusing on transforming lives through technology innovation 4. With over 100+ happy customers and 100+ completed projects, GOKAP has established itself as a trusted name in the tech industry.\n\nFull Name: GOKAP InnoTech (commonly referred to as GIT)\nFounded: 2020\nIndustry: Technology, Freelance Marketplace\nHeadquarters: Thimphu, Bhutan\nContact Information:\nPhone: +97517570958\nEmail: GIT@gokapinnotech.com\nWebsite: www.gokapinnotech.com\n\n2. Platform Description\nGOKAP InnoTech operates a digital marketplace connecting freelancers with clients. The platform facilitates project-based work where

In [87]:
dataset10 = EvaluationDataset(samples=[sample10])

In [88]:
dataset10

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=1)

In [89]:
result10 = evaluate(
    dataset=dataset10, 
    llm=groq_llm,
    embeddings=cohere_embeddings
)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [90]:
result10

{'answer_relevancy': 0.7013, 'context_precision': 1.0000, 'faithfulness': 1.0000, 'context_recall': 1.0000}

## Result

In [92]:
data2 = [result1, result2, result3, result4, result5, result6, result7, result8, result9, result10]

[{'answer_relevancy': 0.8078, 'context_precision': 1.0000, 'faithfulness': 1.0000, 'context_recall': 1.0000},
 {'answer_relevancy': 0.7745, 'context_precision': 1.0000, 'faithfulness': 0.7500, 'context_recall': 0.2500},
 {'answer_relevancy': 0.6403, 'context_precision': 1.0000, 'faithfulness': 0.9167, 'context_recall': 0.7143},
 {'answer_relevancy': 0.7915, 'context_precision': 1.0000, 'faithfulness': 1.0000, 'context_recall': 1.0000},
 {'answer_relevancy': 0.7551, 'context_precision': 0.5000, 'faithfulness': 1.0000, 'context_recall': 1.0000},
 {'answer_relevancy': 0.1716, 'context_precision': 1.0000, 'faithfulness': 1.0000, 'context_recall': 1.0000},
 {'answer_relevancy': 0.7905, 'context_precision': 0.0000, 'faithfulness': 1.0000, 'context_recall': 1.0000},
 {'answer_relevancy': 0.7560, 'context_precision': 1.0000, 'faithfulness': 0.6667, 'context_recall': 1.0000},
 {'answer_relevancy': 0.7246, 'context_precision': 1.0000, 'faithfulness': 1.0000, 'context_recall': 0.2500},
 {'answer_

In [2]:
import pandas as pd

# Data with 'query' as the first field in each dict
data2 = [
    {'query': 'query1', 'answer_relevancy': 0.8078, 'context_precision': 1.0000, 'faithfulness': 1.0000, 'context_recall': 1.0000},
    {'query': 'query2', 'answer_relevancy': 0.7745, 'context_precision': 1.0000, 'faithfulness': 0.7500, 'context_recall': 0.2500},
    {'query': 'query3', 'answer_relevancy': 0.6403, 'context_precision': 1.0000, 'faithfulness': 0.9167, 'context_recall': 0.7143},
    {'query': 'query4', 'answer_relevancy': 0.7915, 'context_precision': 1.0000, 'faithfulness': 1.0000, 'context_recall': 1.0000},
    {'query': 'query5', 'answer_relevancy': 0.7551, 'context_precision': 0.5000, 'faithfulness': 1.0000, 'context_recall': 1.0000},
    {'query': 'query6', 'answer_relevancy': 0.1716, 'context_precision': 1.0000, 'faithfulness': 1.0000, 'context_recall': 1.0000},
    {'query': 'query7', 'answer_relevancy': 0.7905, 'context_precision': 0.0000, 'faithfulness': 1.0000, 'context_recall': 1.0000},
    {'query': 'query8', 'answer_relevancy': 0.7560, 'context_precision': 1.0000, 'faithfulness': 0.6667, 'context_recall': 1.0000},
    {'query': 'query9', 'answer_relevancy': 0.7246, 'context_precision': 1.0000, 'faithfulness': 1.0000, 'context_recall': 0.2500},
    {'query': 'query10', 'answer_relevancy': 0.7013, 'context_precision': 1.0000, 'faithfulness': 1.0000, 'context_recall': 1.0000}
]

df = pd.DataFrame(data2)

print(df)


     query  answer_relevancy  context_precision  faithfulness  context_recall
0   query1            0.8078                1.0        1.0000          1.0000
1   query2            0.7745                1.0        0.7500          0.2500
2   query3            0.6403                1.0        0.9167          0.7143
3   query4            0.7915                1.0        1.0000          1.0000
4   query5            0.7551                0.5        1.0000          1.0000
5   query6            0.1716                1.0        1.0000          1.0000
6   query7            0.7905                0.0        1.0000          1.0000
7   query8            0.7560                1.0        0.6667          1.0000
8   query9            0.7246                1.0        1.0000          0.2500
9  query10            0.7013                1.0        1.0000          1.0000
